In [35]:
from geopy.geocoders import GoogleV3
import numpy as np
import geocoder
import json
import pandas as pd
from geojson import Feature, FeatureCollection, Point
import tabula
import re
import os

### PD Set Options

In [36]:
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', 500)

In [37]:
dsf1 = tabula.read_pdf('Topcondoimage 08-22-2022.pdf', pages='all',lattice=True)
dsf2 = tabula.read_pdf('Topcondoimage 08-08-2022.pdf', pages='all',lattice=True)
dsf3 = tabula.read_pdf('Topcondoimage 08-15-2022.pdf', pages='all',lattice=True)
dsf4 = tabula.read_pdf('Topcondoimage 08-29-2022.pdf', pages='all',lattice=True)
dsf5 = tabula.read_pdf('Topcondoimage 09-06-2022.pdf', pages='all',lattice=True)

In [38]:
df1 = dsf1[0]
df2 = dsf2[0]
df3 = dsf3[0]
df4 = dsf4[0]
df5 = dsf5[0]

In [39]:
df1.columns = df1.iloc[0]
df2.columns = df2.iloc[0]
df3.columns = df3.iloc[0]
df4.columns = df4.iloc[0]
df5.columns = df5.iloc[0]

In [40]:
df1 = df1.drop(df1.index[0])
df2 = df2.drop(df2.index[0])
df3 = df3.drop(df3.index[0])
df4 = df4.drop(df4.index[0])
df5 = df5.drop(df5.index[0])

In [41]:
df1 = df1.iloc[: , 1:]
df2 = df2.iloc[: , 1:]
df3 = df3.iloc[: , 1:]
df4 = df4.iloc[: , 1:]
df5 = df5.iloc[: , 1:]

In [42]:
df = pd.concat([df1, df2, df3, df4, df5], ignore_index=True)

In [43]:
# df = df.replace(np.nan, 'Not Available', regex=True)
df = df.rename(columns={'Building / Address / City': 'building_address_city','Days on\rMarket':'days_on_market',\
                        'Price /\rSq. Ft.':'price_per_sqft'})
df['geo_address'] = df['building_address_city']
df['geo_address'] = df['geo_address'].map(lambda x: re.sub(r'\r', ' ', x))
df = df.replace('\n',' ',regex=True)
df2 = df['geo_address'].str.extract(r'(?P<building_name>\D*)(?P<address_only>\d.*)', expand=True)

Review this later (it's about how to properly merge DFs): https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html

In [44]:
result = pd.merge(df,df2,left_index=True, right_index=True)
df = result

In [45]:
df.columns

Index(['building_address_city', 'List Date', 'Close Date', 'days_on_market',
       'Sale Price', 'Sq. Ft.', 'price_per_sqft', 'Agent', 'Listing Broker',
       'Buyer Agent', 'Buyer Broker', 'geo_address', 'building_name',
       'address_only'],
      dtype='object')

In [46]:
x = df['Close Date'].str.startswith('8')
df = df[x]

In [47]:
len(df)

46

In [48]:
df.columns

Index(['building_address_city', 'List Date', 'Close Date', 'days_on_market',
       'Sale Price', 'Sq. Ft.', 'price_per_sqft', 'Agent', 'Listing Broker',
       'Buyer Agent', 'Buyer Broker', 'geo_address', 'building_name',
       'address_only'],
      dtype='object')

In [49]:
%store -r google_maps_API_Key
geolocator = GoogleV3(api_key=google_maps_API_Key)

In [50]:
df['loc'] = df['address_only'].apply(geolocator.geocode, timeout=10)

/var/folders/rq/0_j_rksd7psgdycg9c1qwf6m0000gp/T/ipykernel_27388/22809927.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['loc'] = df['address_only'].apply(geolocator.geocode, timeout=10)


In [51]:
df['loc']

0                         (Asia, 900 Brickell Key Blvd APT 3103, Miami, FL 33131, USA, (25.7690295, -80.1856979))
1                 (18911 Collins Ave APT 901, Sunny Isles Beach, FL 33160, USA, (25.9509344, -80.11988579999999))
2                               (2301 Collins Ave APT 810, Miami Beach, FL 33139, USA, (25.7987825, -80.1271941))
3                           (300 South Pointe Dr APT 2606, Miami Beach, FL 33139, USA, (25.7677332, -80.1349941))
4                       (19111 Collins Ave APT 3707, Sunny Isles Beach, FL 33160, USA, (25.9534131, -80.1197571))
5                        (15901 Collins Ave APT 406, Sunny Isles Beach, FL 33160, USA, (25.9240235, -80.1218071))
6                       (181 Crandon Blvd APT 205, Key Biscayne, FL 33149, USA, (25.6977458, -80.16075560000002))
7                                   (7330 Ocean Terrace d, Miami Beach, FL 33141, USA, (25.8589938, -80.1201329))
8                          (13647 Deering Bay Dr APT 121, Coral Gables, FL 33158, USA, (

In [52]:
df['point'] = df['loc'].apply(lambda loc: tuple(loc.point) if loc else None)

/var/folders/rq/0_j_rksd7psgdycg9c1qwf6m0000gp/T/ipykernel_27388/1957931448.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['point'] = df['loc'].apply(lambda loc: tuple(loc.point) if loc else None)


Columns must be same length as key explained
- https://cumsum.wordpress.com/2021/06/01/pandas-valueerror-columns-must-be-same-length-as-key/

Creating `df2` based on `df` row indicies.

https://datascience.stackexchange.com/questions/77033/creating-a-new-dataframe-with-specific-row-numbers-from-another

What's tripping this error is when the lat/lon/alititude generator is fed a Nan value to parse - it's effectively trying to split a `null` value into three separate parts - which is impossible.

The code below effectively caputres the anatomy of this error. It creates a new `df` based on a list of indicies from an existing one. Row `56` has no `point` since the `geocoder` couldn't find coordinates for it. The other three do. Splitting the `point` column fails only when row `56` - with `null` values - is attempted.
```
df2 = df.iloc[[56,57,58,59],:]
df2[['lat','lon','altitude']] = pd.DataFrame(df2['point'].to_list(),index=df2.index)
```

In [53]:
df.columns

Index(['building_address_city', 'List Date', 'Close Date', 'days_on_market',
       'Sale Price', 'Sq. Ft.', 'price_per_sqft', 'Agent', 'Listing Broker',
       'Buyer Agent', 'Buyer Broker', 'geo_address', 'building_name',
       'address_only', 'loc', 'point'],
      dtype='object')

In [54]:
df[['lat','lon','altitude']] = pd.DataFrame(df['point'].to_list(),index=df.index)



/var/folders/rq/0_j_rksd7psgdycg9c1qwf6m0000gp/T/ipykernel_27388/3326340165.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[['lat','lon','altitude']] = pd.DataFrame(df['point'].to_list(),index=df.index)
/var/folders/rq/0_j_rksd7psgdycg9c1qwf6m0000gp/T/ipykernel_27388/3326340165.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[['lat','lon','altitude']] = pd.DataFrame(df['point'].to_list(),index=df.index)
/var/folders/rq/0_j_rksd7psgdycg9c1qwf6m0000gp/T/ipykernel_27388/3326340165.py:1: SettingWit

In [55]:
df.index

Int64Index([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
            17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
            34, 35, 36, 37, 38, 39, 40, 42, 45, 46, 48, 49],
           dtype='int64')

In [56]:
### Not sure why this works the way it does. But if I want to simply reset the index because the numbers are askew,
### I need to set both 'drop' and 'inplace' to 'True'
df.reset_index(drop=True, inplace=True)

In [57]:
df

,building_address_city,List Date,Close Date,days_on_market,Sale Price,Sq. Ft.,price_per_sqft,Agent,Listing Broker,Buyer Agent,Buyer Broker,geo_address,building_name,address_only,loc,point,lat,lon,altitude
0,Asia\r900 Brickell Key Blvd 3103\rMiami,5/27/2022,8/18/2022,66,"$4,150,000.00",3398,"$1,221.31",Fabiola Lima,KB Realty,Ana Teresa Rodriguez,Coldwell Banker Realty,Asia 900 Brickell Key Blvd 3103 Miami,Asia,900 Brickell Key Blvd 3103 Miami,"(Asia, 900 Brickell Key Blvd APT 3103, Miami, FL 33131, USA, (25.7690295, -80.1856979))","(25.7690295, -80.1856979, 0.0)",25.769029,-80.185698,0.0
1,Ocean Three Condo\r18911 Collins Ave 901\rSunny Isles Beach,2/25/2022,8/17/2022,112,"$3,200,000.00",3945,$811.15,Madeleyne Sutton,One Sotheby's Int'l Realty,Ekaterina Brosda,"Brosda & Bentley Realtors, LLC",Ocean Three Condo 18911 Collins Ave 901 Sunny Isles Beach,Ocean Three Condo,18911 Collins Ave 901 Sunny Isles Beach,"(18911 Collins Ave APT 901, Sunny Isles Beach, FL 33160, USA, (25.9509344, -80.11988579999999))","(25.9509344, -80.11988579999999, 0.0)",25.950934,-80.119886,0.0
2,Roney Palace Condo\r2301 Collins Ave 810\rMiami Beach,6/9/2022,8/16/2022,8,"$2,475,000.00",1480,"$1,672.30",Ileana De La Torre,Douglas Elliman,Kezia Ceccon,Thine Agency,Roney Palace Condo 2301 Collins Ave 810 Miami Beach,Roney Palace Condo,2301 Collins Ave 810 Miami Beach,"(2301 Collins Ave APT 810, Miami Beach, FL 33139, USA, (25.7987825, -80.1271941))","(25.7987825, -80.1271941, 0.0)",25.798783,-80.127194,0.0
3,Portofino Tower\r300 S Pointe Dr 2606\rMiami Beach,7/8/2022,8/15/2022,6,"$2,200,000.00",2040,"$1,078.43",Christian Brandl,London Foster Realty,Piero Ruggeri,COSMORE Florida Corp.,Portofino Tower 300 S Pointe Dr 2606 Miami Beach,Portofino Tower,300 S Pointe Dr 2606 Miami Beach,"(300 South Pointe Dr APT 2606, Miami Beach, FL 33139, USA, (25.7677332, -80.1349941))","(25.7677332, -80.1349941, 0.0)",25.767733,-80.134994,0.0
4,Ocean Two Condo\r19111 Collins Ave 3707\rSunny Isles Beach,3/5/2022,8/15/2022,82,"$1,562,500.00",2150,$801.28,Gal Kol,Compass Florida LLC,Asi Topaz,"Brokers, LLC",Ocean Two Condo 19111 Collins Ave 3707 Sunny Isles Beach,Ocean Two Condo,19111 Collins Ave 3707 Sunny Isles Beach,"(19111 Collins Ave APT 3707, Sunny Isles Beach, FL 33160, USA, (25.9534131, -80.1197571))","(25.9534131, -80.1197571, 0.0)",25.953413,-80.119757,0.0
5,TDR Tower II Condo\r15901 Collins Ave 406\rSunny Isles Beach,5/25/2022,8/17/2022,72,"$1,500,000.00",2354,$637.21,Giulietta Ulloa,BHHS EWM Realty,Bernard Bell,Sunny Realty,TDR Tower II Condo 15901 Collins Ave 406 Sunny Isles Beach,TDR Tower II Condo,15901 Collins Ave 406 Sunny Isles Beach,"(15901 Collins Ave APT 406, Sunny Isles Beach, FL 33160, USA, (25.9240235, -80.1218071))","(25.9240235, -80.1218071, 0.0)",25.924024,-80.121807,0.0
6,Key Colony Botanica\r181 Crandon Blvd 205\rKey Biscayne,4/20/2022,8/16/2022,47,"$1,480,000.00",1600,$925.00,Dolores Urdapilleta,"Urdapilleta Real Estate, LLC",Istvan Zolcsak,One Sotheby's Int'l Realty,Key Colony Botanica 181 Crandon Blvd 205 Key Biscayne,Key Colony Botanica,181 Crandon Blvd 205 Key Biscayne,"(181 Crandon Blvd APT 205, Key Biscayne, FL 33149, USA, (25.6977458, -80.16075560000002))","(25.6977458, -80.16075560000002, 0.0)",25.697746,-80.160756,0.0
7,St Tropez Condo\r7330 Ocean Ter Ph-D\rMiami Beach,5/18/2022,8/17/2022,49,"$1,450,000.00",1410,"$1,028.37",Enrique Inurrieta,London Foster Realty,Enrique Inurrieta,London Foster Realty,St Tropez Condo 7330 Ocean Ter Ph-D Miami Beach,St Tropez Condo,7330 Ocean Ter Ph-D Miami Beach,"(7330 Ocean Terrace d, Miami Beach, FL 33141, USA, (25.8589938, -80.1201329))","(25.8589938, -80.1201329, 0.0)",25.858994,-80.120133,0.0
8,Venice\r13647 Deering Bay Dr 121\rCoral Gables,5/6/2022,8/15/2022,55,"$1,350,000.00",2750,$490.91,Hazel Goldman,RE/MAX Advance Realty,Lindsey Lamchick,Douglas Elliman,Venice 13647 Deering Bay Dr 121 Coral Gables,Venice,13647 Deering Bay Dr 121 Coral Gables,"(13647 Deering Bay Dr APT 121, Coral Gables, FL 33158, USA,

In [58]:
df.at[9,'building_name']=('Vizicayne South')
df.at[15,'building_name']=('101 Bar Harbour')
df.at[19,'building_name']=('Trump Royale')
df.at[29,'building_name']=('Trump Royale')
df.at[38,'building_name']=('1010 Birckell')
df.at[39,'building_name']=('South Pointe Towers')
df.at[45,'building_name']=('The Fairchild')

df.at[9,'address_only']=('Vizcayne South Condo 253 NE 2nd 4408 Miami')
df.at[19,'address_only']=('Trump Royale 18201 Collins Ave 806 Sunny Isles Beach')
df.at[29,'address_only']=('Trump Royale 18201 Collins Ave 3809 Sunny Isles Beach')
df.at[39,'address_only']=('South Pointe Towers 400 S Pointe Dr 906 Miami Beach')
df.at[45,'address_only']=('The Fairchild 3581 E Glencoe St 408 Miami')

Folium documentation link: https://python-visualization.github.io/folium/index.html

Folium is the library that lets us use leaflet with Python, since it's meant to be used with JavaScript.

For adding titles: https://stackoverflow.com/questions/61928013/adding-a-title-or-text-to-a-folium-map

In [59]:
import folium
from folium.plugins import MarkerCluster

m = folium.Map(location=df[["lat", "lon"]].mean().to_list(), zoom_start=11)
title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'August 2022 Top Miami-Dade Condo Sales ')

# marker_cluster = MarkerCluster().add_to(m)

for i,r in df.iterrows():
    location = (r["lat"], r["lon"])
    info = (r["Sale Price"], r["days_on_market"], r["Agent"], r['Buyer Agent'], r['Sq. Ft.'], r['address_only'], r['building_name'])
    info = list(info)
    new_line = '<br>'
    bold_start = '<strong>'
    bold_end = '</strong>'
    text = f'Building Address: {bold_start}{info[-1]}{bold_end}{new_line}\
    Sale Price: {bold_start}{info[0]}{bold_end}{new_line}Days on Market: \
    {bold_start}{info[1]}{bold_end}{new_line}Listing Agent: {bold_start}{info[2]} \
    {bold_end}{new_line}Buying Agent: {bold_start}{info[3]}{bold_end}{new_line}Square Feet: \
    {bold_start}{info[4]}{bold_end}{new_line}Address: {bold_start}{info[5]}{bold_end}'
    folium.Marker(location=location,
                      tooltip=text)\
    .add_to(m)

m.get_root().html.add_child(folium.Element(title_html))

m

In [60]:
m.save('index.html')

In [61]:
### Clean DF for graphic ###

In [62]:
df['Sale Price int'] = df['Sale Price']
df['price_per_sqft_int'] = df['price_per_sqft']

/var/folders/rq/0_j_rksd7psgdycg9c1qwf6m0000gp/T/ipykernel_27388/2911951518.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Sale Price int'] = df['Sale Price']
/var/folders/rq/0_j_rksd7psgdycg9c1qwf6m0000gp/T/ipykernel_27388/2911951518.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['price_per_sqft_int'] = df['price_per_sqft']


In [63]:
df['Sale Price int'] = df['Sale Price int'].replace('\$','',regex=True)
df['Sale Price int'] = df['Sale Price int'].replace(',','',regex=True)

df['price_per_sqft_int'] = df['price_per_sqft_int'].replace('\$','',regex=True)
df['price_per_sqft_int'] = df['price_per_sqft_int'].replace(',','',regex=True)

/var/folders/rq/0_j_rksd7psgdycg9c1qwf6m0000gp/T/ipykernel_27388/310194361.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Sale Price int'] = df['Sale Price int'].replace('\$','',regex=True)
/var/folders/rq/0_j_rksd7psgdycg9c1qwf6m0000gp/T/ipykernel_27388/310194361.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Sale Price int'] = df['Sale Price int'].replace(',','',regex=True)
/var/folders/rq/0_j_rksd7psgdycg9c1qwf6m0000gp/T/ipykernel_27388/310194361.py:4: SettingWithCopyWarning: 
A value i

In [64]:
df['Sale Price int'] = pd.to_numeric(df['Sale Price int'])
df['price_per_sqft_int'] = pd.to_numeric(df['price_per_sqft_int'])

/var/folders/rq/0_j_rksd7psgdycg9c1qwf6m0000gp/T/ipykernel_27388/879808360.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Sale Price int'] = pd.to_numeric(df['Sale Price int'])
/var/folders/rq/0_j_rksd7psgdycg9c1qwf6m0000gp/T/ipykernel_27388/879808360.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['price_per_sqft_int'] = pd.to_numeric(df['price_per_sqft_int'])


In [65]:
df['days_on_market'] = pd.to_numeric(df['days_on_market'])
df['Sq. Ft.'] = pd.to_numeric(df['Sq. Ft.'])

/var/folders/rq/0_j_rksd7psgdycg9c1qwf6m0000gp/T/ipykernel_27388/1544171665.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['days_on_market'] = pd.to_numeric(df['days_on_market'])
/var/folders/rq/0_j_rksd7psgdycg9c1qwf6m0000gp/T/ipykernel_27388/1544171665.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Sq. Ft.'] = pd.to_numeric(df['Sq. Ft.'])


In [66]:
df.sort_values('Sale Price int',ascending=False)

,building_address_city,List Date,Close Date,days_on_market,Sale Price,Sq. Ft.,price_per_sqft,Agent,Listing Broker,Buyer Agent,Buyer Broker,geo_address,building_name,address_only,loc,point,lat,lon,altitude,Sale Price int,price_per_sqft_int
10,The Muse Residences\r17141 Collins Ave UPH-1\rSunny Isles Beach,11/15/2021,8/3/2022,107.0,"$12,500,000.00",8422,"$2,047.17",Billy Hernandez,Douglas Elliman,Mariam Kaira,Optimar International Realty,The Muse Residences 17141 Collins Ave UPH-1 Sunny Isles Beach,The Muse Residences,17141 Collins Ave UPH-1 Sunny Isles Beach,"(17141 Collins Ave, Sunny Isles Beach, FL 33160, USA, (25.934944, -80.12091819999999))","(25.934944, -80.12091819999999, 0.0)",25.934944,-80.120918,0.0,12500000.0,2047.17
40,Regalia Condo\r19575 Collins Ave 17\rSunny Isles Beach,6/2/2022,8/30/2022,8.0,"$10,700,000.00",4992,"$2,143.43",Jill Hertzberg,Coldwell Banker Realty,Eric Grabois,Envision Realty Group,Regalia Condo 19575 Collins Ave 17 Sunny Isles Beach,Regalia Condo,19575 Collins Ave 17 Sunny Isles Beach,"(19575 Collins Ave UNIT 17, Sunny Isles Beach, FL 33160, USA, (25.956929, -80.1193868))","(25.956929, -80.1193868, 0.0)",25.956929,-80.119387,0.0,10700000.0,2143.43
20,Edition Residences\r2901 Collins Ave 901\rMiami Beach,7/15/2022,8/10/2022,3.0,"$7,200,000.00",2756,"$2,612.48",Marlene Goldman,Beachfront Realty Inc,Roland Ortiz,One Sotheby's INT'L Realty,Edition Residences 2901 Collins Ave 901 Miami Beach,Edition Residences,2901 Collins Ave 901 Miami Beach,"(2901 Collins Ave #901, Miami Beach, FL 33140, USA, (25.8051273, -80.1241113))","(25.8051273, -80.1241113, 0.0)",25.805127,-80.124111,0.0,7200000.0,2612.48
21,Murano at Portofino\r1000 S Pointe Dr 301\rMiami Beach,5/18/2022,8/10/2022,41.0,"$6,300,000.00",3365,"$1,872.21",Dina Goldentayer,Douglas Elliman,Denise Gomm Santos PA,Compass Florida LLC,Murano at Portofino 1000 S Pointe Dr 301 Miami Beach,Murano at Portofino,1000 S Pointe Dr 301 Miami Beach,"(1000 South Pointe Dr APT 301, Miami Beach, FL 33139, USA, (25.7690044, -80.1381107))","(25.7690044, -80.1381107, 0.0)",25.769004,-80.138111,0.0,6300000.0,1872.21
11,One Ocean\r1 Collins Ave 507\rMiami Beach,NaN,8/1/2022,NaN,"$6,300,000.00",3041,"$2,071.69",Comp Sale,Comp Only Sales,Jonathan Corso,Coldwell Banker Realty,One Ocean 1 Collins Ave 507 Miami Beach,One Ocean,1 Collins Ave 507 Miami Beach,"(1 Collins Ave Unit 507, Miami Beach, FL 33139, USA, (25.768079, -80.1339531))","(25.768079, -80.1339531, 0.0)",25.768079,-80.133953,0.0,6300000.0,2071.69
41,Mansions at Acqualina\r17749 Collins Ave 801\rSunny Isles Beach,2/16/2021,8/31/2022,505.0,"$6,250,000.00",4609,"$1,356.04",Denise Rubin,Coldwell Banker Realty,Denise Rubin,Coldwell Banker Realty,Mansions at Acqualina 17749 Collins Ave 801 Sunny Isles Beach,Mansions at Acqualina,17749 Collins Ave 801 Sunny Isles Beach,"(17749 Collins Ave APT 801, Sunny Isles Beach, FL 33160, USA, (25.9400518, -80.12056079999999))","(25.9400518, -80.12056079999999, 0.0)",25.940052,-80.120561,0.0,6250000.0,1356.04
30,Residences by Armani Casa\r18975 Collins Ave 3000\rSunny Isles Beach,11/16/2021,8/22/2022,246.0,"$5,500,000.00",3255,"$1,689.71",Viviane Wolak,One Sotheby's Int'l Realty,Luciene Cofresi PA,Fortune International Realty,Residences by Armani Casa 18975 Collins Ave 3000 Sunny Isles Beach,Residences by Armani Casa,18975 Collins Ave 3000 Sunny Isles Beach,"(18975 Collins Ave #3000, Sunny Isles Beach, FL 33160, USA, (25.9521284, -80.11970810000001))","(25.9521284, -80.11970810000001, 0.0)",25.952128,-80.119708,0.0,5500000.0,1689.71
12,Ocean Tower Three Condo\r781 Crandon Blvd 1204\rKey Biscayne,5/6/2021,8/1/2022,363.0,"$5,000,000.00",2500,"$2,000.00",Monica Ferragut,BHHS EWM Realty,Denise Haas,Gimeza A Real Estate Corp.,Ocean Tower Three Condo 781 Crandon Blvd 1204 Key Biscayne,Ocean Tower Three Condo,781 Crandon Blvd 1204 Key Biscayne,"(781 Crandon Blvd APT 1204, Key Biscayne, FL 33149, USA, (25.6882815, -80.1589302))","(25.6882815, -80.1589302, 0.0)",25.688281,-80.158930,0.0,5000

In [79]:
df.sort_values('days_on_market',ascending=False)

,building_address_city,List Date,Close Date,days_on_market,Sale Price,Sq. Ft.,price_per_sqft,Agent,Listing Broker,Buyer Agent,Buyer Broker,geo_address,building_name,address_only,loc,point,lat,lon,altitude,Sale Price int,price_per_sqft_int
41,Mansions at Acqualina 17749 Collins Ave 801 Sunny Isles Beach,2/16/2021,8/31/2022,505.0,"$6,250,000.00",4609,"$1,356.04",Denise Rubin,Coldwell Banker Realty,Denise Rubin,Coldwell Banker Realty,Mansions at Acqualina 17749 Collins Ave 801 Sunny Isles Beach,Mansions at Acqualina,17749 Collins Ave 801 Sunny Isles Beach,"(17749 Collins Ave APT 801, Sunny Isles Beach, FL 33160, USA, (25.9400518, -80.12056079999999))","(25.9400518, -80.12056079999999, 0.0)",25.940052,-80.120561,0.0,6250000.0,1356.04
15,101 Bar Harbour 10155 Collins Ave 710 Bal Harbour,3/22/2021,8/2/2022,415.0,"$2,495,000.00",3300,$756.06,Carolina Powell,Premier Brokers International,Ryan Mendell,Maxwell E Realty Inc,101 Bar Harbour 10155 Collins Ave 710 Bal Harbour,101 Bar Harbour,101 Bar Harbour 10155 Collins Ave 710 Bal Harbour,"(10155 Collins Ave APT 710, Bal Harbour, FL 33154, USA, (25.8932332, -80.1230573))","(25.8932332, -80.1230573, 0.0)",25.893233,-80.123057,0.0,2495000.0,756.06
12,Ocean Tower Three Condo 781 Crandon Blvd 1204 Key Biscayne,5/6/2021,8/1/2022,363.0,"$5,000,000.00",2500,"$2,000.00",Monica Ferragut,BHHS EWM Realty,Denise Haas,Gimeza A Real Estate Corp.,Ocean Tower Three Condo 781 Crandon Blvd 1204 Key Biscayne,Ocean Tower Three Condo,781 Crandon Blvd 1204 Key Biscayne,"(781 Crandon Blvd APT 1204, Key Biscayne, FL 33149, USA, (25.6882815, -80.1589302))","(25.6882815, -80.1589302, 0.0)",25.688281,-80.158930,0.0,5000000.0,2000.00
13,Ocean Tower Two 791 Crandon Blvd 307 Key Biscayne,8/4/2021,8/1/2022,357.0,"$4,075,000.00",3581,"$1,137.95",Lucia Marin,One Sotheby's INT'L Realty,Antonio Araujo,"Stroomline, LLC.",Ocean Tower Two 791 Crandon Blvd 307 Key Biscayne,Ocean Tower Two,791 Crandon Blvd 307 Key Biscayne,"(791 Crandon Blvd APT 307, Key Biscayne, FL 33149, USA, (25.6883367, -80.1579345))","(25.6883367, -80.1579345, 0.0)",25.688337,-80.157934,0.0,4075000.0,1137.95
26,The Bond 1080 Brickell Ave 3500-3501 Miami,10/12/2021,8/8/2022,303.0,"$1,910,000.00",3051,$700.40,Lida Juliao,Cervera Real Estate Inc.,Robert Himelfarb,Global Luxury Realty LLC,The Bond 1080 Brickell Ave 3500-3501 Miami,The Bond,1080 Brickell Ave 3500-3501 Miami,"(1080 Brickell Ave 3500 3501, Miami, FL 33131, USA, (25.7627976, -80.19265))","(25.7627976, -80.19265, 0.0)",25.762798,-80.192650,0.0,1910000.0,700.40
30,Residences by Armani Casa 18975 Collins Ave 3000 Sunny Isles Beach,11/16/2021,8/22/2022,246.0,"$5,500,000.00",3255,"$1,689.71",Viviane Wolak,One Sotheby's Int'l Realty,Luciene Cofresi PA,Fortune International Realty,Residences by Armani Casa 18975 Collins Ave 3000 Sunny Isles Beach,Residences by Armani Casa,18975 Collins Ave 3000 Sunny Isles Beach,"(18975 Collins Ave #3000, Sunny Isles Beach, FL 33160, USA, (25.9521284, -80.11970810000001))","(25.9521284, -80.11970810000001, 0.0)",25.952128,-80.119708,0.0,5500000.0,1689.71
43,Casa Del Mar Condo 881 Ocean Dr 11H Key Biscayne,11/22/2021,8/30/2022,210.0,"$2,650,000.00",3120,$849.36,Juan Velasquez,Douglas Elliman,Angela Santamaria PA,The Corcoran Group,Casa Del Mar Condo 881 Ocean Dr 11H Key Biscayne,Casa Del Mar Condo,881 Ocean Dr 11H Key Biscayne,"(881 Ocean Dr #11h, Key Biscayne, FL 33149, USA, (25.685686, -80.15729309999999))","(25.685686, -80.15729309999999, 0.0)",25.685686,-80.157293,0.0,2650000.0,849.36
19,T0rump Royale 18201 Collins Ave 806 Sunny Isles Beach,1/10/2022,8/3/2022,192.0,"$1,700,000.00",2137,$795.51,Oscar Donado,Kismet 2 Realty Inc.,Felipe Quintero,EXP Realty LLC,T0rump Royale 18201 Collins Ave 806 Sunny Isles Beach,Trump Royale,Trump Royale 18201 Collins Ave 806 Sunny Isles Beach,"(Trump Royale, 18201 Collins Ave #806, Sunny Isles Beach, FL 33160, USA, (25.9449899, -80.12008929999999))","(25.9449899, -80.12008929999999, 0.0)",25.944990,-80.120089,0.0,1700000.0,795.51
42,M

### Sale price sum

In [68]:
"${:,.2f}".format(df['Sale Price int'].sum())

'$145,356,500.00'

### Sale price average

In [69]:
"${:,.2f}".format(df['Sale Price int'].mean())

'$3,159,923.91'

### Price per sf average

In [70]:
"${:,.2f}".format(df['price_per_sqft_int'].mean())

'$1,260.65'

### Price per square foot min vs. max

In [71]:
"${:,.2f}".format(df['price_per_sqft_int'].max(axis=0))

'$3,015.48'

In [72]:
"${:,.2f}".format(df['price_per_sqft_int'].min(axis=0))

'$490.91'

### Sale price min and max

In [73]:
"${:,.2f}".format(df['Sale Price int'].max(axis=0))

'$12,500,000.00'

In [74]:
"${:,.2f}".format(df['Sale Price int'].min(axis=0))

'$1,250,000.00'

In [75]:
len(df)

46

In [76]:
df['building_address_city'] = df['building_address_city'].map(lambda x: re.sub(r'\r', ' ', x))
df = df.replace('\n',' ',regex=True)
df

/var/folders/rq/0_j_rksd7psgdycg9c1qwf6m0000gp/T/ipykernel_27388/584204004.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['building_address_city'] = df['building_address_city'].map(lambda x: re.sub(r'\r', ' ', x))


,building_address_city,List Date,Close Date,days_on_market,Sale Price,Sq. Ft.,price_per_sqft,Agent,Listing Broker,Buyer Agent,Buyer Broker,geo_address,building_name,address_only,loc,point,lat,lon,altitude,Sale Price int,price_per_sqft_int
0,Asia 900 Brickell Key Blvd 3103 Miami,5/27/2022,8/18/2022,66.0,"$4,150,000.00",3398,"$1,221.31",Fabiola Lima,KB Realty,Ana Teresa Rodriguez,Coldwell Banker Realty,Asia 900 Brickell Key Blvd 3103 Miami,Asia,900 Brickell Key Blvd 3103 Miami,"(Asia, 900 Brickell Key Blvd APT 3103, Miami, FL 33131, USA, (25.7690295, -80.1856979))","(25.7690295, -80.1856979, 0.0)",25.769029,-80.185698,0.0,4150000.0,1221.31
1,Ocean Three Condo 18911 Collins Ave 901 Sunny Isles Beach,2/25/2022,8/17/2022,112.0,"$3,200,000.00",3945,$811.15,Madeleyne Sutton,One Sotheby's Int'l Realty,Ekaterina Brosda,"Brosda & Bentley Realtors, LLC",Ocean Three Condo 18911 Collins Ave 901 Sunny Isles Beach,Ocean Three Condo,18911 Collins Ave 901 Sunny Isles Beach,"(18911 Collins Ave APT 901, Sunny Isles Beach, FL 33160, USA, (25.9509344, -80.11988579999999))","(25.9509344, -80.11988579999999, 0.0)",25.950934,-80.119886,0.0,3200000.0,811.15
2,Roney Palace Condo 2301 Collins Ave 810 Miami Beach,6/9/2022,8/16/2022,8.0,"$2,475,000.00",1480,"$1,672.30",Ileana De La Torre,Douglas Elliman,Kezia Ceccon,Thine Agency,Roney Palace Condo 2301 Collins Ave 810 Miami Beach,Roney Palace Condo,2301 Collins Ave 810 Miami Beach,"(2301 Collins Ave APT 810, Miami Beach, FL 33139, USA, (25.7987825, -80.1271941))","(25.7987825, -80.1271941, 0.0)",25.798783,-80.127194,0.0,2475000.0,1672.30
3,Portofino Tower 300 S Pointe Dr 2606 Miami Beach,7/8/2022,8/15/2022,6.0,"$2,200,000.00",2040,"$1,078.43",Christian Brandl,London Foster Realty,Piero Ruggeri,COSMORE Florida Corp.,Portofino Tower 300 S Pointe Dr 2606 Miami Beach,Portofino Tower,300 S Pointe Dr 2606 Miami Beach,"(300 South Pointe Dr APT 2606, Miami Beach, FL 33139, USA, (25.7677332, -80.1349941))","(25.7677332, -80.1349941, 0.0)",25.767733,-80.134994,0.0,2200000.0,1078.43
4,Ocean Two Condo 19111 Collins Ave 3707 Sunny Isles Beach,3/5/2022,8/15/2022,82.0,"$1,562,500.00",2150,$801.28,Gal Kol,Compass Florida LLC,Asi Topaz,"Brokers, LLC",Ocean Two Condo 19111 Collins Ave 3707 Sunny Isles Beach,Ocean Two Condo,19111 Collins Ave 3707 Sunny Isles Beach,"(19111 Collins Ave APT 3707, Sunny Isles Beach, FL 33160, USA, (25.9534131, -80.1197571))","(25.9534131, -80.1197571, 0.0)",25.953413,-80.119757,0.0,1562500.0,801.28
5,TDR Tower II Condo 15901 Collins Ave 406 Sunny Isles Beach,5/25/2022,8/17/2022,72.0,"$1,500,000.00",2354,$637.21,Giulietta Ulloa,BHHS EWM Realty,Bernard Bell,Sunny Realty,TDR Tower II Condo 15901 Collins Ave 406 Sunny Isles Beach,TDR Tower II Condo,15901 Collins Ave 406 Sunny Isles Beach,"(15901 Collins Ave APT 406, Sunny Isles Beach, FL 33160, USA, (25.9240235, -80.1218071))","(25.9240235, -80.1218071, 0.0)",25.924024,-80.121807,0.0,1500000.0,637.21
6,Key Colony Botanica 181 Crandon Blvd 205 Key Biscayne,4/20/2022,8/16/2022,47.0,"$1,480,000.00",1600,$925.00,Dolores Urdapilleta,"Urdapilleta Real Estate, LLC",Istvan Zolcsak,One Sotheby's Int'l Realty,Key Colony Botanica 181 Crandon Blvd 205 Key Biscayne,Key Colony Botanica,181 Crandon Blvd 205 Key Biscayne,"(181 Crandon Blvd APT 205, Key Biscayne, FL 33149, USA, (25.6977458, -80.16075560000002))","(25.6977458, -80.16075560000002, 0.0)",25.697746,-80.160756,0.0,1480000.0,925.00
7,St Tropez Condo 7330 Ocean Ter Ph-D Miami Beach,5/18/2022,8/17/2022,49.0,"$1,450,000.00",1410,"$1,028.37",Enrique Inurrieta,London Foster Realty,Enrique Inurrieta,London Foster Realty,St Tropez Condo 7330 Ocean Ter Ph-D Miami Beach,St Tropez Condo,7330 Ocean Ter Ph-D Miami Beach,"(7330 Ocean Terrace d, Miami Beach, FL 33141, USA, (25.8589938, -80.1201329))","(25.8589938, -80.1201329, 0.0)",25.858994,-80.120133,0.0,1450000.0,1028.37
8,Venice 13647 Deering Bay Dr 121 Coral Gables,5/6/2022,8/15/2022,55.0,"$1,350,000.00",2750,$490.91,Hazel Goldman,RE/MAX Advance Realty,Lindsey L

## Map URL Snagger

Map template URL: `https://trd-digital.github.io/trd-news-interactive-maps/{map-folder-name}`

In [77]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

In [78]:
cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/CondoSalesAugust2022
